In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import activations
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import scipy as sp

In [3]:
fea_data = np.load('../input/ubiquant-train-data/train_std_data.npy')
# inv_target = np.load('../input/ubiquant-train-data/inv_id.npy')
target_f32 = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')[['investment_id', 'target']]


In [4]:
target_f32['lisandu'] = target_f32.groupby('investment_id')['target'].transform(lambda x: len(x.unique()))

In [5]:
lsd_target = target_f32['lisandu'].values
scaler = MinMaxScaler(feature_range=(1e-3, 1 + 1e-3))
lsd_target = scaler.fit_transform(lsd_target.reshape(-1, 1))
print(lsd_target.min())
print(lsd_target.max())

0.001
1.0009999999999997


In [6]:
# inv_lookup = layers.experimental.preprocessing.IntegerLookup(num_oov_indices=0)
# inv_lookup.adapt(inv_target)
# inv_target = inv_lookup(inv_target).numpy().astype('int16')
# input_dim = fea_data.shape[-1]
# inv_num = len(inv_lookup.get_vocabulary())

# print(input_dim, inv_num)

In [7]:

def build_inv_encoder(input_dim, noise_rate=0.05):
    the_input = Input(shape=(input_dim,), dtype='float32')
    encode = layers.BatchNormalization()(the_input)
    encode = layers.GaussianNoise(noise_rate)(encode)
    encode = layers.Dense(256, activation=activations.swish)(encode)
    decode = layers.Dropout(0.2)(encode)
#     decode = layers.Dense(input_dim, name='decode')(decode)
    x = layers.Dense(256, use_bias=False)(decode)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activations.swish)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1, name='inv_classify')(x)

    inv_encoder = models.Model(inputs=[the_input], outputs=[encode])
    inv_autoencoder = models.Model(inputs=[the_input], outputs=[x])
    inv_autoencoder.compile(optimizer=optimizers.Adam(),
                            loss='mse')
    return inv_autoencoder, inv_encoder


In [8]:
def make_dataset(feature, y, batch_size=512, mode="train", to='mlp'):
    if to == 'mlp':
        ds = tf.data.Dataset.from_tensor_slices((feature, y))
    else:
        ds = tf.data.Dataset.from_tensor_slices((feature, (feature, y)))
    if mode == "train":
        ds = ds.shuffle(4096, seed=145, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [9]:
kfold = KFold(n_splits=6, shuffle=True, random_state=113)
for (train_ind, test_ind) in kfold.split(fea_data, lsd_target):
    train_ds = make_dataset(fea_data[train_ind], lsd_target[train_ind])
    test_ds = make_dataset(fea_data[test_ind], lsd_target[test_ind], mode='valid')
    break
print(train_ds)
print(test_ds)


2022-02-19 19:41:19.038926: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<PrefetchDataset shapes: ((None, 300), (None, 1)), types: (tf.float16, tf.float64)>
<PrefetchDataset shapes: ((None, 300), (None, 1)), types: (tf.float16, tf.float64)>


In [10]:
callback = [callbacks.ReduceLROnPlateau(monitor='val_loss', patience=7, factor=0.55, verbose=1),
            callbacks.EarlyStopping(monitor='val_loss', patience=18, restore_best_weights=True, min_delta=0.001)]

In [11]:
inv_autoencoder, inv_encoder = build_inv_encoder(300)
print(inv_autoencoder.summary())
plot_model(inv_autoencoder, './inv_encoder.png')
inv_autoencoder.fit(train_ds,
                    epochs=50,
                    validation_data=test_ds,
                    callbacks=callback,
                    verbose=1)
inv_encoder.trainable = False
K.clear_session()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 300)               1200      
_________________________________________________________________
gaussian_noise (GaussianNois (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               77056     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65536     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024

2022-02-19 19:41:24.695072: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


5113/5113 [==============================] - 97s 19ms/step - loss: 0.0302 - val_loss: 0.0763
Epoch 2/50
5113/5113 [==============================] - 97s 19ms/step - loss: 0.0115 - val_loss: 0.0366
Epoch 3/50
5113/5113 [==============================] - 95s 19ms/step - loss: 0.0107 - val_loss: 0.0458
Epoch 4/50
5113/5113 [==============================] - 97s 19ms/step - loss: 0.0103 - val_loss: 0.0367
Epoch 5/50
5113/5113 [==============================] - 98s 19ms/step - loss: 0.0100 - val_loss: 0.0407
Epoch 6/50
5113/5113 [==============================] - 98s 19ms/step - loss: 0.0098 - val_loss: 0.0357
Epoch 7/50
5113/5113 [==============================] - 99s 19ms/step - loss: 0.0096 - val_loss: 0.0324
Epoch 8/50
5113/5113 [==============================] - 97s 19ms/step - loss: 0.0095 - val_loss: 0.0334
Epoch 9/50
5113/5113 [==============================] - 97s 19ms/step - loss: 0.0094 - val_loss: 0.0331
Epoch 10/50
5113/5113 [==============================] - 99s 19ms/step - lo